we will train a "vanilla" RNN to predict the sentiment on IMDB reviews. Our data consists of 25000 training sequences and 25000 test sequences. The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers


In [2]:
max_features =2000
maxlen= 30
batch_size= 32


In [3]:
(x_train, y_train), (x_test, y_test)= imdb.load_data(num_words=max_features)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\preay\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\preay\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant

In [4]:
print(len(x_train),"train sequence") 
print(len(x_test),"test sequence")

25000 train sequence
25000 test sequence


In [10]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_train[123,:]

array([ 219,  141,   35,  221,  956,   54,   13,   16,   11,    2,   61,
        322,  423,   12,   38,   76,   59, 1803,   72,    8,    2,   23,
          5,  967,   12,   38,   85,   62,  358,   99])

In [6]:
#here we build RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          100000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 100,286
Trainable params: 100,286
Non-trainable params: 0
_________________________________________________________________


In [11]:
rmsprop = keras.optimizers.RMSprop(lr =0.0001)

model_rnn.compile(loss='binary_crossentropy',
                 optimizer=rmsprop,
                 metrics=['accuracy'])



In [12]:
model_rnn.fit(x_train,y_train,
             batch_size= batch_size,
             epochs=10,
             validation_data=(x_test,y_test))

Epoch 1/10
782/782 [==============================] - 7s 7ms/step - loss: 0.6891 - accuracy: 0.5486 - val_loss: 0.6322 - val_accuracy: 0.6782
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.6062 - accuracy: 0.6895 - val_loss: 0.5652 - val_accuracy: 0.7044
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.5390 - accuracy: 0.7295 - val_loss: 0.5356 - val_accuracy: 0.7248
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.5123 - accuracy: 0.7463 - val_loss: 0.5145 - val_accuracy: 0.7409
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4853 - accuracy: 0.7642 - val_loss: 0.4997 - val_accuracy: 0.7491
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4723 - accuracy: 0.7751 - val_loss: 0.4891 - val_accuracy: 0.7568
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4558 - accuracy: 0.7856 - val_loss: 0.4784 - val_accuracy: 0.7644
Epoch 

In [14]:
score, acc = model_rnn.evaluate(x_test,y_test,batch_size=batch_size)
print("test score:",score)
print("test accuracy", acc)

782/782 [==============================] - 1s 1ms/step - loss: 0.4669 - accuracy: 0.7737
test score: 0.466919481754303
test accuracy 0.7737200260162354
